In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import os
import sys
import jax
import jax.numpy as jnp
# import jax.random
# from gymnasium import spaces
from tqdm import tqdm

from klax import lipschitz_l1_jax, triangular
from rl_environments import Vandelpol, Test2
# from rl_environments import LDSEnv, InvertedPendulum, CollisionAvoidanceEnv, Vandelpol
from rsm_learner import Learner
from rsm_verifier import Verifier
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
parser = argparse.ArgumentParser(
    prog='System Normalization for Neural Certificates',
    description='Learning a reach-avoid supermartingale neural network with normalization techniques',
    epilog='By Musan (Hao Wu) @ SKLCS, Institute of Software, UCAS'
)

parser.add_argument("--debug", default=True, type=bool)
# problem formulation    
# parser.add_argument("--env", default="vandelpol", help='system')
parser.add_argument("--env", default="test2", help='system')
parser.add_argument("--timeout", default=60, type=int, help='max time limit in minutes') 
parser.add_argument("--reach_prob", default=0.8, type=float, help='reach-avoid probability')

parser.add_argument("--project", default=False, type=bool, help='Nomalize the system')

# neural network and training
parser.add_argument("--hidden", default=128, type=int, help='hidden neurons in each layer')
parser.add_argument("--num_layers", default=2, type=int, help='number of hidden layers')
# learner 
parser.add_argument("--continue_rsm", type=int, default=0, help='use an existing network')

# verifier
parser.add_argument("--eps", default=0.01, type=float) # ???
parser.add_argument("--lip", default=0.1, type=float) # ???

 

# parser.add_argument("--p_lip", default=0.0, type=float) 
parser.add_argument("--l_lip", default=10.0, type=float) # ???
parser.add_argument("--fail_check_fast", type=int, default=0)
parser.add_argument("--grid_factor", default=1.0, type=float)
parser.add_argument("--batch_size", default=512, type=int)
# parser.add_argument("--ppo_iters", default=50, type=int)
# parser.add_argument("--policy", default="policies/lds0_zero.jax")
# parser.add_argument("--train_p", type=int, default=1)
parser.add_argument("--square_l_output", default=True)
parser.add_argument("--jitter_grid", type=int, default=0)
parser.add_argument("--soft_constraint", type=int, default=1)
parser.add_argument("--gamma_decrease", default=1.0, type=float)
parser.add_argument("--debug_k0", action="store_true")
parser.add_argument("--gen_plot", action="store_true")
parser.add_argument("--no_refinement", action="store_true")
parser.add_argument("--plot", action="store_true")
parser.add_argument("--small_mem", action="store_true")
sys.argv = ['test.ipynb']
args = parser.parse_args()

In [4]:
# env = Vandelpol()
env = Test2()
learner = Learner(
    env=env,
    l_hidden=[args.hidden] * args.num_layers,
    l_lip=args.l_lip,
    eps=args.eps,
    gamma_decrease=args.gamma_decrease,
    reach_prob=args.reach_prob,
    softplus_l_output=args.square_l_output,
    debug = args.debug,
)

In [5]:
verifier = Verifier(
    learner=learner,
    env=env,
    batch_size=args.batch_size,
    reach_prob=args.reach_prob,
    fail_check_fast=bool(args.fail_check_fast),
    grid_factor=args.grid_factor,
    small_mem=args.small_mem,
    debug = args.debug,
)

In [6]:
from rsm_loop import RSMLoop
loop = RSMLoop( 
        learner,
        verifier,
        env,
        lip_factor=args.lip,
        plot=args.plot,
        jitter_grid=bool(args.jitter_grid),
        soft_constraint=bool(args.soft_constraint),
        debug = args.debug,
    )


loop.plot_l(f"plots/{args.env}_start.png")

In [ ]:
loop.run(args.timeout*60)


## Iteration 0 (0:00 elapsed) ##


Train: loss=1.79, dec_loss=1.53, violations=0.561: 100%|██████████| 100/100 [00:23<00:00,  4.20it/s]


Trained on 40000 samples, start_loss=13.9, end_loss=1.79, start_violations=0.512, end_violations=0.561 in 23.8 seconds


100%|██████████| 77/77 [00:00<00:00, 89.01it/s] 


lipschitz_k=11.152777519226074 (without delta)
delta=0.010050251256281407
K=0.11208821627362889 (with delta)
Checking GRID of size 200


...

lipschitz_k=11.3644246673584 (without delta)
delta=0.010050251256281407
K=0.11421532329003417 (with delta)
Checking GRID of size 200


100%|██████████| 1/1 [00:01<00:00,  1.66s/it]

violations=25371
hard_violations=25371
25371/40000 violated decrease condition
25371/40000 hard violations
Train buffer len: 216660
Grid runtime=2.00 s
info= {'ds_size': 191289, 'lipschitz_k': 11.3644246673584, 'K_f': 1.08, 'K_l': 10.522615432739258, 'iter': 6, 'runtime': 396.7664999961853, 'delta': 0.010050251256281407, 'K': 0.11421532329003417, 'avg_increase': 0, 'dec_violations': '25371/40000', 'hard_violations': '25371/40000'}



## Iteration 7 (8:21 elapsed) ##


Train: loss=0.813, dec_loss=0.622, violations=0.519:  32%|███▏      | 32/100 [00:40<01:32,  1.36s/it]